Class Jeton et Plateau

In [1]:
###Classe representant un jeton caractérisé par sa position en x et y ainsi que sa couleur. Un jeton de joueur sera jaune ou rouge, un jeton blanc est a valeur par défaut
class Jeton:

    def __init__(self, ligne: int= None, colonne: int= None, couleur: str= "blanc"):
        self.ligne = ligne
        self.colonne = colonne
        self.couleur = couleur

    def __str__(self):
        return f"Jeton de couleur {self.couleur} en colonne ({self.colonne} et ligne {self.ligne})"
    
    
    

In [2]:
import numpy as np

###Classe representant le plateau de jeu, par défaut un plateau est rempli de jeton blanc (vide)
class Plateau:

    def __init__(self, historique: list = [], nbLignes: int= 6, nbColonnes: int= 12):
        self.matrice= np.full((nbLignes, nbColonnes), Jeton())
        self.nbLignes = nbLignes
        self.nbColonnes = nbColonnes
        self.historique = historique
        #possibilité de creer un plateau à partir d'une liste de jeton 
        if len(historique) > 0:
            for jeton in historique:
                self.matrice[jeton.ligne][jeton.colonne] = jeton
    
    #méthode permettant l'affichage du plateau     
    def __str__(self):
        
        plateau_str = '\n'
        for ligne in self.matrice[::-1]:
            plateau_str += '|'
            for element in ligne:
                if element.couleur != "blanc": plateau_str += ('\033[91m' + 'O' + '\033[0m' if element.couleur=='rouge' else '\033[93m' + 'O' + '\033[0m') + '|'
                else : plateau_str += ' |'

            plateau_str += '\n'
        
        plateau_str += '-------------------------\n 1 2 3 4 5 6 7 8 9 10 11 12\n'

        return plateau_str
    
    #méthode permettant d'ajouter un jeton au plateau
    def jouerJeton(self, colonne: int, couleur: str):
         for ligne in range(6):
            #si jeton de couleur blanche à la position choisie => case vide donc jouable
            if self.matrice[ligne][colonne].couleur == "blanc":
                jeton = Jeton(ligne, colonne, couleur)
                self.matrice[ligne][colonne] = jeton
                #ajout du jeton jouer à l'historique de jetons joués
                self.historique.append(jeton)
                return ligne

    #méthode retirant le dernier jeton joué
    def annulerCoup(self):
        if len(self.historique) > 0:
            jeton = self.historique.pop()
            self.matrice[jeton.ligne][jeton.colonne] = Jeton()
    
    #méthode pour obtenir le nombre de jeton restant
    def nombreJetons(self):
        return 42-len(self.historique)
    
    #méthode donnant l'état de la partie: 2-> plus de jeton disponible, 1-> partie gagné par un joueur, 0->partie continue, -1-> erreur
    def etat(self, jeton: Jeton):
        
        if len(self.historique)==42:
            return 2 #plateau plein
        else:
          #Pour accelerer le processus de vérification de l'etat nous étudions seulement les alignements comprenant le dernier jeton posé au lieu de parcourir tout le plateau 
            if isinstance(jeton, Jeton):

                #variables qui extrait la ligne, colonne et diagonales qui contiennent le dernier jeton posé
                ligne = self.matrice[jeton.ligne]
                colonne = self.matrice[:, jeton.colonne]
                diagonale1 = np.diagonal(self.matrice, offset=jeton.colonne - jeton.ligne)
                diagonale2 = np.diagonal(np.fliplr(self.matrice), offset=(self.matrice.shape[1]-jeton.colonne-1)-jeton.ligne)

                #utilisation des try/except pour chaques test afin de ne pas avoir à gérer les erreurs d'index (pas le plus optimal mais fonctionnel)

                for i in range(len(ligne)):
                  #vérification de la ligne
                    if ligne[i].couleur != "blanc":
                        try:
                            if ligne[i].couleur == ligne[i+1].couleur == ligne[i+2].couleur == ligne[i+3].couleur:
                                return 1 #gagné
                        except IndexError:
                            pass
                                
                for i in range(len(colonne)):
                  #vérification de la colonne
                    if colonne[i].couleur != "blanc":
                        try:
                            if colonne[i].couleur == colonne[i+1].couleur == colonne[i+2].couleur == colonne[i+3].couleur:
                                return 1 #gagné
                        except IndexError:
                            pass
    
                for i in range(len(diagonale1)):
                  #vérification de la diagonale ascendante
                    if diagonale1[i].couleur != "blanc":
                        try:
                            if diagonale1[i].couleur == diagonale1[i+1].couleur == diagonale1[i+2].couleur == diagonale1[i+3].couleur:
                                return 1 #gagné
                        except IndexError:
                            pass

                for i in range(len(diagonale2)):
                  #vérification de la diagonale descendante
                    if diagonale2[i].couleur != "blanc":
                        try:
                            if diagonale2[i].couleur == diagonale2[i+1].couleur == diagonale2[i+2].couleur == diagonale2[i+3].couleur:
                                return 1 #gagné
                        except IndexError:
                            pass
                                
                return 0 #partie en cours
            else:
                return -1 #erreur
            
 

Class Player et IA

In [3]:
import sys

#classe décrivant un joueur, caractérisé par le fait qu'il soit IA ou non et s'il commence ou non, par défaut le joueur et humain et ne commence pas (jeton jaune)
class Player:
    def __init__(self,start=False, ia=False):
        self.ia=ia
        if start:
            self.couleur='rouge' #les rouges sont les premiers à commencer
        else:
            self.couleur='jaune'
    
    #méthode faisant le processus d'un tour du jeu pour un joueur ou pour l'IA
    def jouer(self, plateau: Plateau):

      #tour pour un joueur
        if not self.ia:
            print("Il reste {} jetons\nJoueur {}, entrez la colonne dans laquelle vous souhaitez jouer (1-12): ".format(plateau.nombreJetons(), self.couleur.upper()), end="")
            
            #entrée de la colonne ou l'on souhaite jouer, avec vérification que l'input soit bien un int entre 1 et 12, demande de réentrer tant que l'input ne respecte pas les conditions
            while True:
                try:
                    colonne= int(input())-1
                    while (colonne<0 or colonne>11 or plateau.matrice[5][colonne].couleur !="blanc") and colonne!=554:
                        print("Colonne invalide, merci de choisir une colonne entre 1 et 12 et non pleine: ")
                        colonne= int(input())-1
                    break
                except:
                    print("Merci de choisir un nombre entre 1 et 12: ", end="")

            #fin de l'exec du prgm
            if colonne==554:
              sys.exit()
            #ajout du jeton dans la colonne choisie 
            plateau.jouerJeton(colonne, self.couleur)
            #affichage du plateau
            print(plateau)

        #tour pour l'IA
        else:
            print("L'IA joue...\n", end="")
            #coup de l'IA
            iaJouerJeton(plateau, self.couleur)
            print(plateau)


In [4]:
import time

###Fonctions relatives à l'IA

#Algo min max avec elagage alpha beta
def minimax_alpha_beta(plateau: Plateau, couleur: str, couleurIA: str, depth: int, maximizingPlayer: bool, alpha: float, beta: float,depthMax = 5):

    # Vérifie si on est arrivé à la profondeur maximale ou si le jeu est terminé
    try:
        etat= plateau.etat(plateau.historique[-1])
    except:
        etat= plateau.etat(None)

    if etat == 1:
        scoreFinal = 100000 / (depthMax+1-depth)
        valeur = -scoreFinal if maximizingPlayer else scoreFinal
        return valeur
    if etat == 2:
        print("plateau plein")
        return 0
    if depth == 0:
        valeur = heuristicScore(plateau, couleurIA)

        return valeur

    # Maximizing player
    if maximizingPlayer:
        value = float("-inf")

        for colonne in coupsPossibles(plateau):
            plateau.jouerJeton(colonne, couleur)
            value = max(value,minimax_alpha_beta(plateau, getCouleurAdverse(couleur), couleurIA, depth-1, False, alpha, beta,depthMax))
            plateau.annulerCoup()
            alpha = max(alpha, value)
            if beta <= value:
                break  # élagage Alpha-Beta
        return value

    # Minimizing player
    else:
        value = float("inf")

        for colonne in coupsPossibles(plateau):
            plateau.jouerJeton (colonne, couleur)
            value = min(value,minimax_alpha_beta(plateau, getCouleurAdverse(couleur), couleurIA, depth-1, True, alpha, beta,depthMax))
            plateau.annulerCoup()
            beta = min(beta, value)
            if value <= alpha:
                break  # élagage Alpha-Beta
        return value

#methode retournant une liste des colonnes ou il est possible de jouer
def coupsPossibles(plateau: Plateau):
    coupsPossibles=[]
    for i in range(12):
        if plateau.matrice[5][i].couleur=="blanc":
            coupsPossibles.append(i)
    return coupsPossibles

#methode retournant la couleur des jetons de l'adversaire
def getCouleurAdverse(couleur: str):
    return 'jaune' if couleur=='rouge' else 'rouge'

#methode calculant l'heuristique
def heuristicScore(plateau: Plateau, couleurIA: str):
    multiplicateur = [1, 5]
    score = ptsVertical(plateau, couleurIA, multiplicateur) + \
        ptsHorizontal(plateau, couleurIA, multiplicateur) + \
        ptsDiagonale(plateau, couleurIA, multiplicateur)
    return score

#generateur de point:
#on compte sur tout le plateau le nombre de groupe de 2 ou 3 jetons alignés, une fois ces nombres trouvés on applique un coef multiplicateur (1 pour les groupes de 2 et 5 pour les grp de 3)

def ptsVertical(plateau: Plateau, couleurIA: str, multiplicateur: int):
    inARow = 0
    trois = 0
    deux = 0
    for i in range(12):
        for j in range(6):
                if plateau.matrice[j][i].couleur == couleurIA:
                    inARow += 1
                else:
                    inARow = 0
                if inARow == 3:
                    trois += 1
                    deux -= 1
                elif inARow == 2:
                    deux += 1
        inARow = 0
    return deux * multiplicateur[0] + trois * multiplicateur[1]

def ptsHorizontal(plateau: Plateau, couleurIA: str, multiplicateur: int):
    inARow = 0
    trois = 0
    deux = 0
    for i in range(6):
        for j in range(12):
                if plateau.matrice[i][j].couleur == couleurIA:
                    inARow += 1
                else:
                    inARow = 0
                if inARow == 3:
                    trois += 1
                    deux -= 1
                elif inARow == 2:
                    deux += 1
        inARow = 0
    return deux * multiplicateur[0] + trois * multiplicateur[1]

def ptsDiagonale(plateau: Plateau, couleurIA: str, multiplicateur: int):
    inARow = [0, 0, 0, 0]
    trois = 0
    deux = 0
    for i in range(5):
        for j in range(6):
                if j+i < 6:
                    if plateau.matrice[j+i][j].couleur == couleurIA:
                        inARow[0] += 1
                    else:
                        inARow[0] = 0
                    if plateau.matrice[j+i][12 - 1 - j].couleur == couleurIA:
                        inARow[2] += 1
                    else:
                        inARow[2] = 0
                if j+i < 12:
                    if plateau.matrice[j][j+i].couleur == couleurIA:
                        inARow[1] += 1
                    else:
                        inARow[1] = 0
                    if plateau.matrice[j][12 - 1 - j-i].couleur == couleurIA:
                        inARow[3] += 1
                    else:
                        inARow[3] = 0
                for r in inARow:
                    if r == 3:
                        trois += 1
                        deux -= 1
                    elif r == 2:
                        deux += 1
        inARow = [0, 0, 0, 0]

    return deux * multiplicateur[0] + trois * multiplicateur[1]

#décorateur calculant le temps d'execution d'une fonction
def calculTempsExec(fonction):
    def inner(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        print("Temps d'exécution: {} secondes".format(fin - debut))
        return resultat
    return inner


#méthode executant le processus de jeu de l'IA
@calculTempsExec
def iaJouerJeton(plateau, couleur: str):
    bestColonne = 0
    bestScore = float("-inf")

    #calcul du meilleur coup grace à l'algo min max
    for coup in coupsPossibles(plateau):
        plateau.jouerJeton(coup, couleur)
        #appel de l'algo
        score = minimax_alpha_beta(plateau, getCouleurAdverse(couleur), couleur, 4, False,float('-inf'), float('inf'), 5)
        plateau.annulerCoup()
        if score > bestScore:
            bestScore = score
            bestColonne = coup

    #ajout du jeton de l'IA sur le plateau
    plateau.jouerJeton(bestColonne, couleur)
    print("Colonne jouée par l'IA: {}".format(bestColonne+1))
    return 0
 

Main

In [5]:
import sys

if __name__== '__main__':
    plateau = Plateau()

    # Choix du mode de jeu avec sécurité sur l'input
    while True:
        try:
            modeJeu= int(input("Mode de jeu: 1. Joueur contre joueur, 2. Joueur contre IA\n"))
            while modeJeu!=1 and modeJeu!=2 and modeJeu!=555:
                print("Merci de choisir un mode de jeu valide: ")
                modeJeu= int(input())
            break
        except:
            print("Merci de choisir entre 1 et 2: ")

    #fin de l'exec du prgm
    if modeJeu==555:
      sys.exit()

    elif modeJeu==1: # Joueur contre joueur
        joueur1 = Player(start=True)
        joueur2 = Player()
    
    else: # Joueur contre IA

        # choix du joueur qui commence avec sécurité sur l'input
        while True: 
            try:
                start= int(input("Qui commence: 1. Joueur, 2. IA\n"))
                while start!=1 and start!=2 and start!=555:
                    print("Merci de choisir un chiffre valide: ")
                    modeJeu= int(input())
                break
            except:
                print("Merci de choisir entre 1 et 2: ")
        
        #fin de l'exec du prgm
        if start==555:
          sys.exit()

        if start==1: #joueur 1 humain et 2 IA
            joueur1 = Player(start=True)
            joueur2 = Player(ia=True)
        else: #joueur 1 IA et 2 humain
            joueur1 = Player(start=True, ia=True)
            joueur2 = Player()

    while True: # Boucle de jeu
        joueur1.jouer(plateau)
        if plateau.etat(plateau.historique[-1]) == 2:
            print("Match nul !")
            break
        elif plateau.etat(plateau.historique[-1]) == 1:
            print("Le joueur {} a gagné !".format(joueur1.couleur.upper()))
            break

        joueur2.jouer(plateau)
        if plateau.etat(plateau.historique[-1]) == 1:
            print("Le joueur {} a gagné !".format(joueur2.couleur.upper()))
            break
        elif plateau.etat(plateau.historique[-1]) == 2:
            print("Match nul !")
            break

Mode de jeu: 1. Joueur contre joueur, 2. Joueur contre IA
2
Qui commence: 1. Joueur, 2. IA
2
L'IA joue...
Colonne jouée par l'IA: 2
Temps d'exécution: 4.774643421173096 secondes

| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| |O| | | | | | | | | | |
-------------------------
 1 2 3 4 5 6 7 8 9 10 11 12

Il reste 41 jetons
Joueur JAUNE, entrez la colonne dans laquelle vous souhaitez jouer (1-12): 3

| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| |O|O| | | | | | | | | |
-------------------------
 1 2 3 4 5 6 7 8 9 10 11 12

L'IA joue...
Colonne jouée par l'IA: 2
Temps d'exécution: 3.645136594772339 secondes

| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| | | | | | | | | | | | |
| |O| | | | | | | | | | |
| |O|O| | | | | | | | | |
-------------------------
 1 2 3 4 5 6 7 8 9 10 11 12

Il re